In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
#--# 문제1
import pandas as pd
import sklearn.model_selection
#--# 문제4
import gymnasium as gym
import IPython
from matplotlib.animation import FuncAnimation
import collections
import random

# 3. `Human_Numbers`

`HUMAN_NUMBERS_train.txt`  사람이 읽을 수 있는 형식으로 숫자가 나열된 텍스트 파일이다. 이 텍스트 파일을 이용하여 “현재 단어가 주어졌을 때 다음 단어를 예측하는” 신경망을 설계하고 학습시킬 것이다. 아래는 해당 파일을 불러와 단어 단위로 나눈 결과를 확인하는 코드이다. 

In [3]:
!wget https://raw.githubusercontent.com/guebin/DL2025/main/posts/HUMAN_NUMBERS_train.txt
with open('HUMAN_NUMBERS_train.txt') as f:
    words = f.read().split()

--2025-06-18 06:32:13--  https://raw.githubusercontent.com/guebin/DL2025/main/posts/HUMAN_NUMBERS_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... failed: Connection timed out.
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 280599 (274K) [text/plain]
Saving to: ‘HUMAN_NUMBERS_train.txt’

HUMAN_NUMBERS_train 100%[===================>] 274.02K  --.-KB/s    in 0.02s   

2025-06-18 06:34:23 (16.0 MB/s) - ‘HUMAN_NUMBERS_train.txt’ saved [280599/280599]



In [2]:
with open('HUMAN_NUMBERS_train.txt') as f:
    words = f.read().split()

In [3]:
print(words[-12:])

['seven', 'thousand', 'nine', 'hundred', 'ninety', 'eight', 'seven', 'thousand', 'nine', 'hundred', 'ninety', 'nine']


이 텍스트 파일에는 예를 들어 "... seven, thousand, nine, hundred, ninety, eight, ..." 등과 같은 사람이 읽는 숫자 표현이 단어 단위로 나열된 시퀀스 형태로 저장되어 있다.

 이러한 데이터에서 "현재 단어가 주어졌을 때 다음 단어를 예측하는" 자연어 처리 모델을 학습하라. 즉 아래와 같은 맵핑을 학습하라. (꼭 다맞출 필요는 없습니다)

- ...  seven, thousand, nine, hundred, ninety, eight $\to$ seven
- ...  seven, thousand, nine, hundred, ninety, eight, seven $\to$ thousand
- ...  seven, thousand, nine, hundred, ninety, eight, seven, thousand $\to$ nine 

**제약사항**

- one-hot 전처리 코드를 포함할 것
- `torch.nn.RNN`, `torch.nn.RNNCell`, `torch.nn.LSTM` 중 하나를 이용할 것 
- 처음 12개의 단어와  마지막 12개의 단어에 대한 적합값(fitted value)을 제시할 것 

**hint**: 아래의 코드를 이용하여 전처리하면 편리하다. 

In [4]:
df_train = pd.DataFrame({'x': words[:-1], 'y': words[1:]})
df_train

,x,y
0,one,two
1,two,three
2,three,four
3,four,five
4,five,six
...,...,...
42074,seven,thousand
42075,thousand,nine
42076,nine,hundred
42077,hundred,ninety


In [5]:
# 전체 vocab 기준으로 맵핑
vocab = sorted(set(words))
dct= {w: i for i, w in enumerate(vocab)}
# train 데이터셋
x = torch.tensor(df_train.x.map(dct))
y = torch.tensor(df_train.y.map(dct))

In [6]:
vocab

['eight',
 'eighteen',
 'eighty',
 'eleven',
 'fifteen',
 'fifty',
 'five',
 'forty',
 'four',
 'fourteen',
 'hundred',
 'nine',
 'nineteen',
 'ninety',
 'one',
 'seven',
 'seventeen',
 'seventy',
 'six',
 'sixteen',
 'sixty',
 'ten',
 'thirteen',
 'thirty',
 'thousand',
 'three',
 'twelve',
 'twenty',
 'two']

In [7]:
len(vocab)

29

In [8]:
dct

{'eight': 0,
 'eighteen': 1,
 'eighty': 2,
 'eleven': 3,
 'fifteen': 4,
 'fifty': 5,
 'five': 6,
 'forty': 7,
 'four': 8,
 'fourteen': 9,
 'hundred': 10,
 'nine': 11,
 'nineteen': 12,
 'ninety': 13,
 'one': 14,
 'seven': 15,
 'seventeen': 16,
 'seventy': 17,
 'six': 18,
 'sixteen': 19,
 'sixty': 20,
 'ten': 21,
 'thirteen': 22,
 'thirty': 23,
 'thousand': 24,
 'three': 25,
 'twelve': 26,
 'twenty': 27,
 'two': 28}

- one-hot 전처리 코드

In [10]:
X = torch.nn.functional.one_hot(x).float()
y = torch.nn.functional.one_hot(y).float()

In [11]:
X.shape, y.shape

(torch.Size([42079, 29]), torch.Size([42079, 29]))

## 기존 풀이 (틀림)

- torch.nn.LSTM

In [12]:
torch.manual_seed(5)
lstm = torch.nn.LSTM(
    input_size  = 29, # X.shape = (L,29)
    hidden_size = 2, # h.shape = (L,2)
    num_layers = 1,  # num_layers = 1 이라는 의미 
    bidirectional = False # D=1 이라는 의미 
)    
cook = torch.nn.Linear(2,29)
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(
    list(lstm.parameters())+
    list(cook.parameters()),
    lr=0.1
)
#---#
for epoc in range(200):
    # 1
    h, _ = lstm(X) # rnn의 결과는 길이가 2인 튜플이고, 첫원소가 h 
    netout = cook(h)
    # 2 
    loss = loss_fn(netout, y)
    # 3
    loss.backward()
    # 4 
    optimizr.step()
    optimizr.zero_grad()

In [13]:
yhat = torch.nn.functional.softmax(netout,dim=1)

- 처음 12개의 단어와 마지막 12개의 단어에 대한 적합값(fitted value)을 제시

In [14]:
yhat

tensor([[1.5848e-05, 6.8493e-06, 3.4633e-07,  ..., 1.9942e-06, 5.1928e-07,
         9.1639e-04],
        [1.8921e-05, 5.4279e-06, 2.6273e-07,  ..., 1.5076e-06, 3.9237e-07,
         1.0290e-03],
        [2.6468e-05, 7.7439e-06, 4.1175e-07,  ..., 2.2037e-06, 6.1075e-07,
         1.3581e-03],
        ...,
        [2.0989e-04, 2.3373e-04, 5.2029e-05,  ..., 1.0565e-04, 7.0884e-05,
         5.5811e-03],
        [9.7104e-03, 7.8019e-03, 9.7290e-02,  ..., 8.9866e-03, 9.7032e-02,
         1.6517e-02],
        [1.1783e-01, 7.6676e-04, 5.3032e-03,  ..., 4.9830e-04, 5.0763e-03,
         1.0248e-01]], grad_fn=<SoftmaxBackward0>)

In [15]:
yhat.shape

torch.Size([42079, 29])

In [16]:
# 1) 예측 인덱스 & 단어
pred_idx   = yhat.argmax(dim=1).cpu().numpy()           # (L,)
pred_words = [vocab[i] for i in pred_idx]               # 예측 단어 리스트

In [17]:
# 2) 결과 DataFrame 생성
df_out = pd.DataFrame({
    'true':  df_train['y'].values,  # 실제 단어
    'pred':  pred_words            # 예측 단어
})

In [18]:
# 3) 처음 12개와 마지막 12개 추출·출력
print("=== First 12 Predictions ===")
print(df_out.head(12))

print("\n=== Last 12 Predictions ===")
print(df_out.tail(12))

=== First 12 Predictions ===
        true      pred
0        two  thousand
1      three  thousand
2       four  thousand
3       five  thousand
4        six  thousand
5      seven  thousand
6      eight  thousand
7       nine      four
8        ten   hundred
9     eleven      four
10    twelve   hundred
11  thirteen  thousand

=== Last 12 Predictions ===
           true      pred
42067     seven   hundred
42068  thousand  thousand
42069      nine      four
42070   hundred   hundred
42071    ninety   seventy
42072     eight      nine
42073     seven       one
42074  thousand  thousand
42075      nine      four
42076   hundred   hundred
42077    ninety   seventy
42078      nine      nine


In [19]:
# 1) 청크 번호 생성
chunk_size = 1000
df_out['chunk'] = np.arange(len(df_out)) // chunk_size

# 2) 맞혔는지 여부 컬럼 추가
df_out['correct'] = (df_out['true'] == df_out['pred'])

# 3) 청크별 정확도(accuracy) 계산
accuracy_per_chunk = (
    df_out
    .groupby('chunk', as_index=False)['correct']
    .mean()
    .rename(columns={'correct': 'accuracy'})
)

print(accuracy_per_chunk)

    chunk  accuracy
0       0  0.076000
1       1  0.087000
2       2  0.077000
3       3  0.216000
4       4  0.395000
5       5  0.462000
6       6  0.425000
7       7  0.400000
8       8  0.396000
9       9  0.334000
10     10  0.361000
11     11  0.469000
12     12  0.372000
13     13  0.369000
14     14  0.333000
15     15  0.365000
16     16  0.431000
17     17  0.403000
18     18  0.373000
19     19  0.364000
20     20  0.417000
21     21  0.383000
22     22  0.496000
23     23  0.397000
24     24  0.377000
25     25  0.351000
26     26  0.365000
27     27  0.429000
28     28  0.406000
29     29  0.372000
30     30  0.366000
31     31  0.329000
32     32  0.362000
33     33  0.472000
34     34  0.372000
35     35  0.366000
36     36  0.333000
37     37  0.363000
38     38  0.428000
39     39  0.410000
40     40  0.371000
41     41  0.367000
42     42  0.341772


- 적합률이 많이 떨어짐
- 이후 GPT4o를 활용하여 다른 모델들도 실행하여 제출했으나 생략
    - 적합률이 대부분 떨어지고 높더라도 모르는 내용이 많음

## 보완 풀이 1 (hidden_size=5)

- input size가 29이기 때문에 hidden_size를 5로 수정
    - 29 < 2^5라고 생각 

- torch.nn.LSTM

In [20]:
torch.manual_seed(5)
lstm = torch.nn.LSTM(
    input_size  = 29, # X.shape = (L,29)
    hidden_size = 5, # h.shape = (L,5)
    num_layers = 1,  # num_layers = 1 이라는 의미 
    bidirectional = False # D=1 이라는 의미 
)    
cook = torch.nn.Linear(5,29)
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(
    list(lstm.parameters())+
    list(cook.parameters()),
    lr=0.1
)
#---#
for epoc in range(200):
    # 1
    h, _ = lstm(X) # rnn의 결과는 길이가 2인 튜플이고, 첫원소가 h 
    netout = cook(h)
    # 2 
    loss = loss_fn(netout, y)
    # 3
    loss.backward()
    # 4 
    optimizr.step()
    optimizr.zero_grad()

In [21]:
yhat = torch.nn.functional.softmax(netout,dim=1)

In [22]:
yhat

tensor([[1.4474e-03, 5.3983e-04, 5.0411e-03,  ..., 1.7794e-04, 4.3711e-03,
         1.0791e-02],
        [1.8573e-03, 3.2795e-03, 5.9997e-03,  ..., 2.0096e-04, 4.4850e-03,
         5.3760e-03],
        [1.9884e-04, 8.0082e-04, 2.0528e-03,  ..., 4.2291e-05, 1.9412e-03,
         8.5825e-04],
        ...,
        [9.6103e-05, 3.5775e-06, 1.4959e-04,  ..., 9.4313e-06, 1.2673e-04,
         7.5691e-04],
        [3.3314e-03, 4.9775e-03, 1.0756e-01,  ..., 1.1231e-02, 1.1286e-01,
         3.3395e-03],
        [9.1997e-02, 1.1108e-05, 1.1170e-04,  ..., 3.3530e-05, 1.4796e-04,
         1.0327e-01]], grad_fn=<SoftmaxBackward0>)

In [23]:
yhat.shape

torch.Size([42079, 29])

In [24]:
# 1) 예측 인덱스 & 단어
pred_idx   = yhat.argmax(dim=1).cpu().numpy()           # (L,)
pred_words = [vocab[i] for i in pred_idx]               # 예측 단어 리스트

In [25]:
# 2) 결과 DataFrame 생성
df_out = pd.DataFrame({
    'true':  df_train['y'].values,  # 실제 단어
    'pred':  pred_words            # 예측 단어
})

In [26]:
# 3) 처음 12개와 마지막 12개 추출·출력
print("=== First 12 Predictions ===")
print(df_out.head(12))

print("\n=== Last 12 Predictions ===")
print(df_out.tail(12))

=== First 12 Predictions ===
        true      pred
0        two   hundred
1      three  thousand
2       four  thousand
3       five  thousand
4        six  thousand
5      seven  thousand
6      eight  thousand
7       nine   hundred
8        ten   hundred
9     eleven      four
10    twelve      four
11  thirteen  thousand

=== Last 12 Predictions ===
           true      pred
42067     seven      four
42068  thousand  thousand
42069      nine       one
42070   hundred   hundred
42071    ninety     fifty
42072     eight      four
42073     seven      four
42074  thousand  thousand
42075      nine      four
42076   hundred   hundred
42077    ninety    twenty
42078      nine      five


In [27]:
# 1) 청크 번호 생성
chunk_size = 1000
df_out['chunk'] = np.arange(len(df_out)) // chunk_size

# 2) 맞혔는지 여부 컬럼 추가
df_out['correct'] = (df_out['true'] == df_out['pred'])

# 3) 청크별 정확도(accuracy) 계산
accuracy_per_chunk = (
    df_out
    .groupby('chunk', as_index=False)['correct']
    .mean()
    .rename(columns={'correct': 'accuracy'})
)

print(accuracy_per_chunk)

    chunk  accuracy
0       0  0.318000
1       1  0.290000
2       2  0.294000
3       3  0.358000
4       4  0.456000
5       5  0.512000
6       6  0.484000
7       7  0.459000
8       8  0.456000
9       9  0.379000
10     10  0.399000
11     11  0.480000
12     12  0.417000
13     13  0.403000
14     14  0.385000
15     15  0.425000
16     16  0.437000
17     17  0.429000
18     18  0.396000
19     19  0.417000
20     20  0.432000
21     21  0.452000
22     22  0.543000
23     23  0.475000
24     24  0.450000
25     25  0.411000
26     26  0.476000
27     27  0.456000
28     28  0.461000
29     29  0.423000
30     30  0.460000
31     31  0.377000
32     32  0.397000
33     33  0.463000
34     34  0.415000
35     35  0.393000
36     36  0.367000
37     37  0.438000
38     38  0.425000
39     39  0.434000
40     40  0.395000
41     41  0.419000
42     42  0.367089


- 비슷함
- seed를 0,1로도 해봤으나 마찬가지 결과

## 보완 풀이 2 (역방향 추가)

- 문자열이 복잡하므로 역방향 학습을 해보기로 함
    - bidrectional = True 설정 후
    - 그에 맞춰 cook도 수정 

- torch.nn.LSTM

In [ ]:
torch.manual_seed(0)
lstm = torch.nn.LSTM(
    input_size  = 29, # X.shape = (L,29)
    hidden_size = 5, # h.shape = (L,5)
    num_layers = 1,  # num_layers = 1 이라는 의미 
    bidirectional = True # D=2 이라는 의미 
)    
cook = torch.nn.Linear(10,29)
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(
    list(lstm.parameters())+
    list(cook.parameters()),
    lr=0.1
)
#---#
for epoc in range(200):
    # 1
    h, _ = lstm(X) # rnn의 결과는 길이가 2인 튜플이고, 첫원소가 h 
    netout = cook(h)
    # 2 
    loss = loss_fn(netout, y)
    # 3
    loss.backward()
    # 4 
    optimizr.step()
    optimizr.zero_grad()

In [46]:
yhat = torch.nn.functional.softmax(netout,dim=1)

- 처음 12개의 단어와 마지막 12개의 단어에 대한 적합값(fitted value)을 제시

In [47]:
yhat

tensor([[3.1617e-06, 2.0103e-09, 3.7032e-05,  ..., 1.8721e-10, 2.3848e-09,
         1.4301e-03],
        [7.9249e-04, 1.1029e-08, 5.2581e-11,  ..., 3.5054e-13, 1.0932e-08,
         3.0752e-06],
        [3.7814e-08, 4.1972e-11, 5.7738e-08,  ..., 1.1455e-10, 2.0236e-11,
         6.3567e-03],
        ...,
        [2.2288e-06, 1.8809e-12, 7.1074e-06,  ..., 1.1688e-13, 3.0204e-05,
         1.4058e-12],
        [2.0933e-06, 1.1628e-05, 2.6661e-01,  ..., 3.3115e-04, 7.5557e-03,
         1.1487e-07],
        [6.4547e-01, 7.1928e-04, 8.5033e-05,  ..., 5.0513e-08, 2.3217e-01,
         1.1310e-08]], grad_fn=<SoftmaxBackward0>)

In [48]:
yhat.shape

torch.Size([42079, 29])

In [49]:
# 1) 예측 인덱스 & 단어
pred_idx   = yhat.argmax(dim=1).cpu().numpy()           # (L,)
pred_words = [vocab[i] for i in pred_idx]               # 예측 단어 리스트

In [50]:
# 2) 결과 DataFrame 생성
df_out = pd.DataFrame({
    'true':  df_train['y'].values,  # 실제 단어
    'pred':  pred_words            # 예측 단어
})

In [51]:
# 3) 처음 12개와 마지막 12개 추출·출력
print("=== First 12 Predictions ===")
print(df_out.head(12))

print("\n=== Last 12 Predictions ===")
print(df_out.tail(12))

=== First 12 Predictions ===
        true      pred
0        two  thousand
1      three     three
2       four  thousand
3       five      five
4        six  thousand
5      seven       one
6      eight     eight
7       nine   hundred
8        ten    eleven
9     eleven   hundred
10    twelve     fifty
11  thirteen   sixteen

=== Last 12 Predictions ===
           true      pred
42067     seven     seven
42068  thousand  thousand
42069      nine      nine
42070   hundred   hundred
42071    ninety    ninety
42072     eight     eight
42073     seven     seven
42074  thousand  thousand
42075      nine      nine
42076   hundred   hundred
42077    ninety    ninety
42078      nine     eight


In [52]:
# 1) 청크 번호 생성
chunk_size = 1000
df_out['chunk'] = np.arange(len(df_out)) // chunk_size

# 2) 맞혔는지 여부 컬럼 추가
df_out['correct'] = (df_out['true'] == df_out['pred'])

# 3) 청크별 정확도(accuracy) 계산
accuracy_per_chunk = (
    df_out
    .groupby('chunk', as_index=False)['correct']
    .mean()
    .rename(columns={'correct': 'accuracy'})
)

print(accuracy_per_chunk)

    chunk  accuracy
0       0  0.851000
1       1  0.966000
2       2  0.927000
3       3  0.929000
4       4  0.999000
5       5  0.997000
6       6  0.997000
7       7  0.997000
8       8  0.999000
9       9  0.975000
10     10  0.995000
11     11  0.995000
12     12  0.993000
13     13  0.994000
14     14  0.970000
15     15  0.992000
16     16  0.985000
17     17  0.985000
18     18  0.985000
19     19  0.993000
20     20  0.969000
21     21  1.000000
22     22  1.000000
23     23  0.999000
24     24  1.000000
25     25  0.976000
26     26  0.995000
27     27  0.993000
28     28  0.995000
29     29  0.994000
30     30  0.994000
31     31  0.987000
32     32  0.993000
33     33  0.993000
34     34  0.992000
35     35  0.996000
36     36  0.987000
37     37  0.996000
38     38  0.997000
39     39  0.996000
40     40  0.997000
41     41  0.996000
42     42  0.987342


- 처음 1,000개의 row 이후에는 정확도가 매우 높음
- seed를 변경해서 한 번 더 시도

## 보완 풀이 3 (역방향 추가 + seed 변경)

- torch.nn.LSTM

In [53]:
torch.manual_seed(1)
lstm = torch.nn.LSTM(
    input_size  = 29, # X.shape = (L,29)
    hidden_size = 5, # h.shape = (L,5)
    num_layers = 1,  # num_layers = 1 이라는 의미 
    bidirectional = True # D=2 이라는 의미 
)    
cook = torch.nn.Linear(10,29)
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(
    list(lstm.parameters())+
    list(cook.parameters()),
    lr=0.1
)
#---#
for epoc in range(200):
    # 1
    h, _ = lstm(X) # rnn의 결과는 길이가 2인 튜플이고, 첫원소가 h 
    netout = cook(h)
    # 2 
    loss = loss_fn(netout, y)
    # 3
    loss.backward()
    # 4 
    optimizr.step()
    optimizr.zero_grad()

In [54]:
yhat = torch.nn.functional.softmax(netout,dim=1)

- 처음 12개의 단어와 마지막 12개의 단어에 대한 적합값(fitted value)을 제시

In [55]:
yhat

tensor([[1.6349e-02, 3.6277e-08, 5.8893e-04,  ..., 4.8575e-08, 3.0039e-16,
         1.3193e-01],
        [2.5609e-03, 4.3839e-12, 1.0113e-10,  ..., 1.2924e-05, 2.7538e-11,
         1.5459e-08],
        [3.6096e-02, 4.4106e-05, 9.0622e-04,  ..., 4.1877e-03, 2.4234e-10,
         3.8882e-03],
        ...,
        [1.3680e-07, 1.7353e-07, 5.3064e-09,  ..., 2.5751e-09, 1.9787e-06,
         1.0270e-14],
        [1.3613e-08, 9.9616e-05, 8.3639e-10,  ..., 4.1368e-06, 2.1724e-04,
         4.3472e-19],
        [3.6704e-08, 3.8188e-10, 3.3055e-12,  ..., 1.4791e-10, 5.8062e-04,
         2.9436e-16]], grad_fn=<SoftmaxBackward0>)

In [56]:
yhat.shape

torch.Size([42079, 29])

In [57]:
# 1) 예측 인덱스 & 단어
pred_idx   = yhat.argmax(dim=1).cpu().numpy()           # (L,)
pred_words = [vocab[i] for i in pred_idx]               # 예측 단어 리스트

In [58]:
# 2) 결과 DataFrame 생성
df_out = pd.DataFrame({
    'true':  df_train['y'].values,  # 실제 단어
    'pred':  pred_words            # 예측 단어
})

In [59]:
# 3) 처음 12개와 마지막 12개 추출·출력
print("=== First 12 Predictions ===")
print(df_out.head(12))

print("\n=== Last 12 Predictions ===")
print(df_out.tail(12))

=== First 12 Predictions ===
        true       pred
0        two       five
1      three      three
2       four       four
3       five        two
4        six        six
5      seven    hundred
6      eight      eight
7       nine    hundred
8        ten    hundred
9     eleven        six
10    twelve        six
11  thirteen  seventeen

=== Last 12 Predictions ===
           true      pred
42067     seven     seven
42068  thousand  thousand
42069      nine      nine
42070   hundred   hundred
42071    ninety    ninety
42072     eight     eight
42073     seven     seven
42074  thousand  thousand
42075      nine      nine
42076   hundred   hundred
42077    ninety     forty
42078      nine      nine


In [60]:
# 1) 청크 번호 생성
chunk_size = 1000
df_out['chunk'] = np.arange(len(df_out)) // chunk_size

# 2) 맞혔는지 여부 컬럼 추가
df_out['correct'] = (df_out['true'] == df_out['pred'])

# 3) 청크별 정확도(accuracy) 계산
accuracy_per_chunk = (
    df_out
    .groupby('chunk', as_index=False)['correct']
    .mean()
    .rename(columns={'correct': 'accuracy'})
)

print(accuracy_per_chunk)

    chunk  accuracy
0       0  0.877000
1       1  0.980000
2       2  0.963000
3       3  0.956000
4       4  0.998000
5       5  0.995000
6       6  0.996000
7       7  0.996000
8       8  0.998000
9       9  0.974000
10     10  0.994000
11     11  0.994000
12     12  0.994000
13     13  0.996000
14     14  0.972000
15     15  0.999000
16     16  0.998000
17     17  0.998000
18     18  0.998000
19     19  0.998000
20     20  0.973000
21     21  0.998000
22     22  0.997000
23     23  0.998000
24     24  0.999000
25     25  0.972000
26     26  0.996000
27     27  0.993000
28     28  0.994000
29     29  0.994000
30     30  0.997000
31     31  0.976000
32     32  0.990000
33     33  0.991000
34     34  0.990000
35     35  0.995000
36     36  0.970000
37     37  0.996000
38     38  0.993000
39     39  0.995000
40     40  0.996000
41     41  0.994000
42     42  0.987342


- 마찬가지로 처음 1,000개의 row 이후에는 정확도가 매우 높음

## 보완 풀이 4 (hidden_size=6)

- input_size는 29이지만 문맥에 따라 사실 그 이상이 될 여지가 충분하므로
- hidden_size를 높여봄
- 역방향은 하지 않음 (bidrectional = False)

- torch.nn.LSTM

In [61]:
torch.manual_seed(5)
lstm = torch.nn.LSTM(
    input_size  = 29, # X.shape = (L,29)
    hidden_size = 6, # h.shape = (L,6)
    num_layers = 1,  # num_layers = 1 이라는 의미 
    bidirectional = False # D=1 이라는 의미 
)    
cook = torch.nn.Linear(6,29)
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(
    list(lstm.parameters())+
    list(cook.parameters()),
    lr=0.1
)
#---#
for epoc in range(200):
    # 1
    h, _ = lstm(X) # rnn의 결과는 길이가 2인 튜플이고, 첫원소가 h 
    netout = cook(h)
    # 2 
    loss = loss_fn(netout, y)
    # 3
    loss.backward()
    # 4 
    optimizr.step()
    optimizr.zero_grad()

In [62]:
yhat = torch.nn.functional.softmax(netout,dim=1)

- 처음 12개의 단어와 마지막 12개의 단어에 대한 적합값(fitted value)을 제시

In [63]:
yhat

tensor([[3.7065e-05, 4.5527e-03, 1.5230e-02,  ..., 6.1987e-03, 1.5407e-02,
         5.7883e-05],
        [3.3132e-03, 3.2297e-04, 1.6524e-04,  ..., 4.1582e-04, 1.2672e-04,
         1.2493e-02],
        [7.6287e-05, 4.6955e-04, 1.6344e-03,  ..., 6.4444e-04, 1.7983e-03,
         2.9418e-04],
        ...,
        [5.5518e-05, 7.8149e-06, 1.5539e-04,  ..., 1.1828e-05, 7.7259e-05,
         4.0750e-04],
        [5.9013e-04, 3.7120e-03, 1.2060e-01,  ..., 3.3190e-03, 1.2618e-01,
         2.4807e-04],
        [9.5842e-02, 6.6577e-05, 2.1009e-05,  ..., 6.2041e-05, 2.2768e-05,
         1.1822e-01]], grad_fn=<SoftmaxBackward0>)

In [64]:
yhat.shape

torch.Size([42079, 29])

In [65]:
# 1) 예측 인덱스 & 단어
pred_idx   = yhat.argmax(dim=1).cpu().numpy()           # (L,)
pred_words = [vocab[i] for i in pred_idx]               # 예측 단어 리스트

In [66]:
# 2) 결과 DataFrame 생성
df_out = pd.DataFrame({
    'true':  df_train['y'].values,  # 실제 단어
    'pred':  pred_words            # 예측 단어
})

In [67]:
# 3) 처음 12개와 마지막 12개 추출·출력
print("=== First 12 Predictions ===")
print(df_out.head(12))

print("\n=== Last 12 Predictions ===")
print(df_out.tail(12))

=== First 12 Predictions ===
        true      pred
0        two   hundred
1      three   hundred
2       four  thousand
3       five  thousand
4        six  thousand
5      seven  thousand
6      eight  thousand
7       nine   hundred
8        ten   hundred
9     eleven   hundred
10    twelve   hundred
11  thirteen      four

=== Last 12 Predictions ===
           true      pred
42067     seven       two
42068  thousand  thousand
42069      nine       two
42070   hundred   hundred
42071    ninety    twenty
42072     eight       two
42073     seven     three
42074  thousand  thousand
42075      nine       two
42076   hundred   hundred
42077    ninety    twenty
42078      nine       two


In [68]:
# 1) 청크 번호 생성
chunk_size = 1000
df_out['chunk'] = np.arange(len(df_out)) // chunk_size

# 2) 맞혔는지 여부 컬럼 추가
df_out['correct'] = (df_out['true'] == df_out['pred'])

# 3) 청크별 정확도(accuracy) 계산
accuracy_per_chunk = (
    df_out
    .groupby('chunk', as_index=False)['correct']
    .mean()
    .rename(columns={'correct': 'accuracy'})
)

print(accuracy_per_chunk)

    chunk  accuracy
0       0  0.132000
1       1  0.063000
2       2  0.132000
3       3  0.324000
4       4  0.453000
5       5  0.460000
6       6  0.405000
7       7  0.410000
8       8  0.408000
9       9  0.432000
10     10  0.548000
11     11  0.482000
12     12  0.469000
13     13  0.482000
14     14  0.415000
15     15  0.547000
16     16  0.550000
17     17  0.525000
18     18  0.517000
19     19  0.478000
20     20  0.378000
21     21  0.475000
22     22  0.406000
23     23  0.391000
24     24  0.389000
25     25  0.357000
26     26  0.443000
27     27  0.434000
28     28  0.387000
29     29  0.390000
30     30  0.399000
31     31  0.359000
32     32  0.475000
33     33  0.393000
34     34  0.389000
35     35  0.381000
36     36  0.357000
37     37  0.437000
38     38  0.433000
39     39  0.387000
40     40  0.387000
41     41  0.388000
42     42  0.341772


- 실패

## 보완 풀이 5 (hidden_size=7)

- hidden_size만 7로 높여서 실행

- torch.nn.LSTM

In [69]:
torch.manual_seed(5)
lstm = torch.nn.LSTM(
    input_size  = 29, # X.shape = (L,29)
    hidden_size = 7, # h.shape = (L,7)
    num_layers = 1,  # num_layers = 1 이라는 의미 
    bidirectional = False # D=1 이라는 의미 
)    
cook = torch.nn.Linear(7,29)
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(
    list(lstm.parameters())+
    list(cook.parameters()),
    lr=0.1
)
#---#
for epoc in range(200):
    # 1
    h, _ = lstm(X) # rnn의 결과는 길이가 2인 튜플이고, 첫원소가 h 
    netout = cook(h)
    # 2 
    loss = loss_fn(netout, y)
    # 3
    loss.backward()
    # 4 
    optimizr.step()
    optimizr.zero_grad()

In [70]:
yhat = torch.nn.functional.softmax(netout,dim=1)

- 처음 12개의 단어와 마지막 12개의 단어에 대한 적합값(fitted value)을 제시

In [71]:
yhat

tensor([[1.8283e-07, 1.1450e-07, 1.4956e-03,  ..., 6.6693e-08, 4.0494e-05,
         1.5115e-05],
        [1.0970e-02, 7.1388e-05, 1.1522e-01,  ..., 5.6457e-05, 1.2502e-02,
         1.4366e-02],
        [1.8247e-09, 3.0444e-09, 7.2681e-04,  ..., 2.7827e-09, 1.8954e-06,
         1.8903e-07],
        ...,
        [6.2995e-11, 1.3118e-11, 4.2953e-05,  ..., 1.4996e-11, 4.1832e-08,
         7.4277e-09],
        [1.1162e-04, 7.2878e-04, 1.3145e-01,  ..., 4.0824e-04, 1.1171e-01,
         3.5104e-04],
        [8.5766e-02, 1.2777e-06, 4.3670e-08,  ..., 2.0657e-06, 1.4005e-05,
         1.1935e-01]], grad_fn=<SoftmaxBackward0>)

In [72]:
yhat.shape

torch.Size([42079, 29])

In [73]:
# 1) 예측 인덱스 & 단어
pred_idx   = yhat.argmax(dim=1).cpu().numpy()           # (L,)
pred_words = [vocab[i] for i in pred_idx]               # 예측 단어 리스트

In [74]:
# 2) 결과 DataFrame 생성
df_out = pd.DataFrame({
    'true':  df_train['y'].values,  # 실제 단어
    'pred':  pred_words            # 예측 단어
})

In [75]:
# 3) 처음 12개와 마지막 12개 추출·출력
print("=== First 12 Predictions ===")
print(df_out.head(12))

print("\n=== Last 12 Predictions ===")
print(df_out.tail(12))

=== First 12 Predictions ===
        true      pred
0        two  thousand
1      three     fifty
2       four   hundred
3       five    eighty
4        six  thousand
5      seven  thousand
6      eight     sixty
7       nine  thousand
8        ten  thousand
9     eleven      four
10    twelve      four
11  thirteen      four

=== Last 12 Predictions ===
           true      pred
42067     seven       one
42068  thousand  thousand
42069      nine      nine
42070   hundred   hundred
42071    ninety     fifty
42072     eight       two
42073     seven       one
42074  thousand  thousand
42075      nine      nine
42076   hundred   hundred
42077    ninety    eighty
42078      nine       two


In [76]:
# 1) 청크 번호 생성
chunk_size = 1000
df_out['chunk'] = np.arange(len(df_out)) // chunk_size

# 2) 맞혔는지 여부 컬럼 추가
df_out['correct'] = (df_out['true'] == df_out['pred'])

# 3) 청크별 정확도(accuracy) 계산
accuracy_per_chunk = (
    df_out
    .groupby('chunk', as_index=False)['correct']
    .mean()
    .rename(columns={'correct': 'accuracy'})
)

print(accuracy_per_chunk)

    chunk  accuracy
0       0  0.300000
1       1  0.412000
2       2  0.346000
3       3  0.410000
4       4  0.436000
5       5  0.480000
6       6  0.473000
7       7  0.480000
8       8  0.585000
9       9  0.379000
10     10  0.405000
11     11  0.398000
12     12  0.401000
13     13  0.473000
14     14  0.444000
15     15  0.404000
16     16  0.408000
17     17  0.403000
18     18  0.423000
19     19  0.524000
20     20  0.381000
21     21  0.424000
22     22  0.409000
23     23  0.411000
24     24  0.498000
25     25  0.437000
26     26  0.409000
27     27  0.419000
28     28  0.397000
29     29  0.429000
30     30  0.527000
31     31  0.373000
32     32  0.424000
33     33  0.411000
34     34  0.411000
35     35  0.457000
36     36  0.480000
37     37  0.468000
38     38  0.476000
39     39  0.473000
40     40  0.489000
41     41  0.541000
42     42  0.544304


- 또 실패

## 보완 풀이 6 (hidden_size=6, 역방향 추가)

- hidden_size = 6
- 역방향 추가 (bidrectional = True)

- torch.nn.LSTM

In [93]:
torch.manual_seed(5)
lstm = torch.nn.LSTM(
    input_size  = 29, # X.shape = (L,29)
    hidden_size = 6, # h.shape = (L,6)
    num_layers = 1,  # num_layers = 1 이라는 의미 
    bidirectional = True # D=2 이라는 의미 
)    
cook = torch.nn.Linear(12,29)
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(
    list(lstm.parameters())+
    list(cook.parameters()),
    lr=0.1
)
#---#
for epoc in range(200):
    # 1
    h, _ = lstm(X) # rnn의 결과는 길이가 2인 튜플이고, 첫원소가 h 
    netout = cook(h)
    # 2 
    loss = loss_fn(netout, y)
    # 3
    loss.backward()
    # 4 
    optimizr.step()
    optimizr.zero_grad()

In [94]:
yhat = torch.nn.functional.softmax(netout,dim=1)

- 처음 12개의 단어와 마지막 12개의 단어에 대한 적합값(fitted value)을 제시

In [95]:
yhat

tensor([[1.2533e-07, 2.8298e-09, 9.9293e-11,  ..., 1.8811e-10, 5.2931e-07,
         7.5078e-04],
        [9.9083e-07, 4.0537e-10, 1.2277e-06,  ..., 7.8233e-10, 1.4352e-05,
         8.4817e-03],
        [2.9894e-11, 4.9861e-09, 3.4209e-02,  ..., 3.1657e-09, 1.7631e-03,
         9.1169e-09],
        ...,
        [9.6490e-09, 4.9059e-09, 7.7631e-05,  ..., 1.5136e-07, 1.1839e-05,
         1.3791e-10],
        [1.4970e-12, 2.9719e-03, 5.2750e-08,  ..., 3.6019e-05, 2.0028e-05,
         3.0589e-13],
        [1.1729e-02, 5.5592e-03, 2.1014e-09,  ..., 1.0461e-02, 7.6911e-09,
         2.5942e-05]], grad_fn=<SoftmaxBackward0>)

In [96]:
yhat.shape

torch.Size([42079, 29])

In [97]:
# 1) 예측 인덱스 & 단어
pred_idx   = yhat.argmax(dim=1).cpu().numpy()           # (L,)
pred_words = [vocab[i] for i in pred_idx]               # 예측 단어 리스트

In [98]:
# 2) 결과 DataFrame 생성
df_out = pd.DataFrame({
    'true':  df_train['y'].values,  # 실제 단어
    'pred':  pred_words            # 예측 단어
})

In [99]:
# 3) 처음 12개와 마지막 12개 추출·출력
print("=== First 12 Predictions ===")
print(df_out.head(12))

print("\n=== Last 12 Predictions ===")
print(df_out.tail(12))

=== First 12 Predictions ===
        true      pred
0        two  thousand
1      three  thousand
2       four   hundred
3       five      five
4        six  thousand
5      seven     seven
6      eight  thousand
7       nine      nine
8        ten       ten
9     eleven   sixteen
10    twelve     eight
11  thirteen  thirteen

=== Last 12 Predictions ===
           true      pred
42067     seven     seven
42068  thousand  thousand
42069      nine      nine
42070   hundred   hundred
42071    ninety    ninety
42072     eight     eight
42073     seven     seven
42074  thousand  thousand
42075      nine      nine
42076   hundred   hundred
42077    ninety    ninety
42078      nine       six


In [100]:
# 1) 청크 번호 생성
chunk_size = 1000
df_out['chunk'] = np.arange(len(df_out)) // chunk_size

# 2) 맞혔는지 여부 컬럼 추가
df_out['correct'] = (df_out['true'] == df_out['pred'])

# 3) 청크별 정확도(accuracy) 계산
accuracy_per_chunk = (
    df_out
    .groupby('chunk', as_index=False)['correct']
    .mean()
    .rename(columns={'correct': 'accuracy'})
)

print(accuracy_per_chunk)

    chunk  accuracy
0       0  0.932000
1       1  0.998000
2       2  0.997000
3       3  0.960000
4       4  1.000000
5       5  1.000000
6       6  1.000000
7       7  1.000000
8       8  1.000000
9       9  0.991000
10     10  1.000000
11     11  1.000000
12     12  1.000000
13     13  1.000000
14     14  0.993000
15     15  1.000000
16     16  1.000000
17     17  1.000000
18     18  0.999000
19     19  1.000000
20     20  0.995000
21     21  1.000000
22     22  0.999000
23     23  0.998000
24     24  1.000000
25     25  0.991000
26     26  1.000000
27     27  1.000000
28     28  1.000000
29     29  1.000000
30     30  1.000000
31     31  0.990000
32     32  1.000000
33     33  1.000000
34     34  1.000000
35     35  1.000000
36     36  0.993000
37     37  1.000000
38     38  1.000000
39     39  1.000000
40     40  1.000000
41     41  1.000000
42     42  0.987342


- hidden_size = 5에서 보이지 않았던 정확도 1도 보임

## 보완 풀이 7 (hidden_size=7, 역방향 추가)

- 이미 잘 되지만 hidden_size도 7까지 올림

- torch.nn.LSTM

In [101]:
torch.manual_seed(5)
lstm = torch.nn.LSTM(
    input_size  = 29, # X.shape = (L,29)
    hidden_size = 7, # h.shape = (L,7)
    num_layers = 1,  # num_layers = 1 이라는 의미 
    bidirectional = True # D=2 이라는 의미 
)    
cook = torch.nn.Linear(14,29)
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(
    list(lstm.parameters())+
    list(cook.parameters()),
    lr=0.1
)
#---#
for epoc in range(200):
    # 1
    h, _ = lstm(X) # rnn의 결과는 길이가 2인 튜플이고, 첫원소가 h 
    netout = cook(h)
    # 2 
    loss = loss_fn(netout, y)
    # 3
    loss.backward()
    # 4 
    optimizr.step()
    optimizr.zero_grad()

In [102]:
yhat = torch.nn.functional.softmax(netout,dim=1)

- 처음 12개의 단어와 마지막 12개의 단어에 대한 적합값(fitted value)을 제시

In [103]:
yhat

tensor([[4.7526e-08, 8.7862e-07, 5.0891e-08,  ..., 2.2394e-06, 8.4799e-08,
         2.8584e-02],
        [4.0320e-09, 1.3876e-03, 3.3420e-05,  ..., 2.3096e-05, 3.2566e-07,
         3.9486e-04],
        [5.9579e-13, 3.8629e-07, 4.2675e-07,  ..., 4.7583e-10, 2.9717e-11,
         2.7420e-14],
        ...,
        [1.1023e-06, 1.7860e-08, 1.1023e-10,  ..., 3.5089e-09, 5.4323e-05,
         5.0075e-04],
        [2.3697e-04, 1.6339e-06, 3.5264e-03,  ..., 1.3583e-06, 2.6748e-03,
         5.1017e-06],
        [5.4513e-01, 1.5457e-10, 9.3180e-07,  ..., 1.4596e-06, 1.1473e-06,
         4.5226e-01]], grad_fn=<SoftmaxBackward0>)

In [104]:
yhat.shape

torch.Size([42079, 29])

In [105]:
# 1) 예측 인덱스 & 단어
pred_idx   = yhat.argmax(dim=1).cpu().numpy()           # (L,)
pred_words = [vocab[i] for i in pred_idx]               # 예측 단어 리스트

In [106]:
# 2) 결과 DataFrame 생성
df_out = pd.DataFrame({
    'true':  df_train['y'].values,  # 실제 단어
    'pred':  pred_words            # 예측 단어
})

In [107]:
# 3) 처음 12개와 마지막 12개 추출·출력
print("=== First 12 Predictions ===")
print(df_out.head(12))

print("\n=== Last 12 Predictions ===")
print(df_out.tail(12))

=== First 12 Predictions ===
        true      pred
0        two   hundred
1      three   hundred
2       four      four
3       five      five
4        six  thousand
5      seven  thousand
6      eight  thousand
7       nine      four
8        ten  thousand
9     eleven  fourteen
10    twelve     eight
11  thirteen  thirteen

=== Last 12 Predictions ===
           true      pred
42067     seven     seven
42068  thousand  thousand
42069      nine      nine
42070   hundred   hundred
42071    ninety    ninety
42072     eight     eight
42073     seven     seven
42074  thousand  thousand
42075      nine      nine
42076   hundred   hundred
42077    ninety    ninety
42078      nine     eight


In [108]:
# 1) 청크 번호 생성
chunk_size = 1000
df_out['chunk'] = np.arange(len(df_out)) // chunk_size

# 2) 맞혔는지 여부 컬럼 추가
df_out['correct'] = (df_out['true'] == df_out['pred'])

# 3) 청크별 정확도(accuracy) 계산
accuracy_per_chunk = (
    df_out
    .groupby('chunk', as_index=False)['correct']
    .mean()
    .rename(columns={'correct': 'accuracy'})
)

print(accuracy_per_chunk)

    chunk  accuracy
0       0  0.925000
1       1  1.000000
2       2  0.991000
3       3  0.993000
4       4  1.000000
5       5  1.000000
6       6  1.000000
7       7  1.000000
8       8  1.000000
9       9  0.992000
10     10  1.000000
11     11  1.000000
12     12  1.000000
13     13  1.000000
14     14  0.990000
15     15  1.000000
16     16  1.000000
17     17  1.000000
18     18  1.000000
19     19  1.000000
20     20  0.999000
21     21  1.000000
22     22  1.000000
23     23  1.000000
24     24  1.000000
25     25  0.999000
26     26  1.000000
27     27  1.000000
28     28  1.000000
29     29  1.000000
30     30  1.000000
31     31  0.997000
32     32  1.000000
33     33  1.000000
34     34  1.000000
35     35  1.000000
36     36  0.996000
37     37  1.000000
38     38  1.000000
39     39  1.000000
40     40  1.000000
41     41  1.000000
42     42  0.987342


- 마찬가지로 잘 됨
- 정확도 1도 많음